In [1]:
class RoundRobin:
    def __init__(self, servers):
        self.servers = servers
        self.current_index = -1

    def get_next_server(self):
        self.current_index = (self.current_index + 1) % len(self.servers)
        return self.servers[self.current_index]

# Example usage
servers = ["Server1", "Server2", "Server3"]
load_balancer = RoundRobin(servers)

for i in range(6):
    server = load_balancer.get_next_server()
    print(f"Request {i + 1} -> {server}")

Request 1 -> Server1
Request 2 -> Server2
Request 3 -> Server3
Request 4 -> Server1
Request 5 -> Server2
Request 6 -> Server3


In [3]:
import hashlib

class IPHash():
    def __init__(self, servers):
        self.servers = servers

    def get_next_server(self, client_ip):
        hash_value = hashlib.md5(client_ip.encode()).hexdigest()
        print("hash_value",hash_value)
        index = int(hash_value, 16) % len(self.servers)
        print("index", index)
        return self.servers[index]

# Example usage
servers = ["Server1", "Server2", "Server3"]
load_balancer = IPHash(servers)

client_ips = ["192.168.0.1", "192.168.0.2", "192.168.0.3", "192.168.0.4"]
for ip in client_ips:
    server = load_balancer.get_next_server(ip)
    print(f"Client {ip} -> {server}")

hash_value f0fdb4c3f58e3e3f8e77162d893d3055
index 2
Client 192.168.0.1 -> Server3
hash_value 6f65e91667cf98dd13464deaf2739fde
index 2
Client 192.168.0.2 -> Server3
hash_value 8c27f6b5dc0e9e5a536aa0371648fb4a
index 0
Client 192.168.0.3 -> Server1
hash_value 0252f5158dc3a9531907fb42c00a4138
index 2
Client 192.168.0.4 -> Server3


In [4]:
import random

class LeastConnections:
    def __init__(self, servers):
        self.servers = {server: 0 for server in servers}

    def get_next_server(self):
        # Find the minimum number of connections
        min_connections = min(self.servers.values())
        print("min",min_connections)
        # Get all servers with the minimum number of connections
        least_loaded_servers = [server for server, connections in self.servers.items() if connections == min_connections]
        print("least", least_loaded_servers)
        # Select a random server from the least loaded servers
        selected_server = random.choice(least_loaded_servers)
        print("selected", selected_server)
        self.servers[selected_server] += 1
        return selected_server

    def release_connection(self, server):
        if self.servers[server] > 0:
            self.servers[server] -= 1

# Example usage
servers = ["Server1", "Server2", "Server3"]
load_balancer = LeastConnections(servers)

for i in range(6):
    server = load_balancer.get_next_server()
    print(f"Request {i + 1} -> {server}")
    load_balancer.release_connection(server)

min 0
least ['Server1', 'Server2', 'Server3']
selected Server1
Request 1 -> Server1
min 0
least ['Server1', 'Server2', 'Server3']
selected Server3
Request 2 -> Server3
min 0
least ['Server1', 'Server2', 'Server3']
selected Server1
Request 3 -> Server1
min 0
least ['Server1', 'Server2', 'Server3']
selected Server3
Request 4 -> Server3
min 0
least ['Server1', 'Server2', 'Server3']
selected Server2
Request 5 -> Server2
min 0
least ['Server1', 'Server2', 'Server3']
selected Server3
Request 6 -> Server3


In [6]:
import time
import random

class LeastResponseTime:
    def __init__(self, servers):
        self.servers = servers
        self.response_times = [0] * len(servers)

    def get_next_server(self):
        min_response_time = min(self.response_times)
        print("min_response_time",min_response_time)
        min_index = self.response_times.index(min_response_time)
        print("min_index",min_index)
        return self.servers[min_index]

    def update_response_time(self, server, response_time):
        index = self.servers.index(server)
        print("index", index)
        self.response_times[index] = response_time

# Simulated server response time function
def simulate_response_time():
    # Simulating response time with random delay
    delay = random.uniform(0.1, 1.0)
    print("delay", delay)
    time.sleep(delay)
    return delay

# Example usage
servers = ["Server1", "Server2", "Server3"]
load_balancer = LeastResponseTime(servers)

for i in range(6):
    server = load_balancer.get_next_server()
    print(f"Request {i + 1} -> {server}")
    response_time = simulate_response_time()
    load_balancer.update_response_time(server, response_time)
    print(f"Response Time: {response_time:.2f}s")

min_response_time 0
min_index 0
Request 1 -> Server1
delay 0.6133780765440447
index 0
Response Time: 0.61s
min_response_time 0
min_index 1
Request 2 -> Server2
delay 0.9601032800952441
index 1
Response Time: 0.96s
min_response_time 0
min_index 2
Request 3 -> Server3
delay 0.4913419286336643
index 2
Response Time: 0.49s
min_response_time 0.4913419286336643
min_index 2
Request 4 -> Server3
delay 0.15140194137645652
index 2
Response Time: 0.15s
min_response_time 0.15140194137645652
min_index 2
Request 5 -> Server3
delay 0.12889841294875837
index 2
Response Time: 0.13s
min_response_time 0.12889841294875837
min_index 2
Request 6 -> Server3
delay 0.7438610149418212
index 2
Response Time: 0.74s


In [7]:
class WeightedRoundRobin:
    def __init__(self, servers, weights):
        self.servers = servers
        self.weights = weights
        self.current_index = -1
        self.current_weight = 0

    def get_next_server(self):
        while True:
            self.current_index = (self.current_index + 1) % len(self.servers)
            if self.current_index == 0:
                self.current_weight -= 1
                if self.current_weight <= 0:
                    self.current_weight = max(self.weights)
            if self.weights[self.current_index] >= self.current_weight:
                return self.servers[self.current_index]

# Example usage
servers = ["Server1", "Server2", "Server3"]
weights = [5, 1, 1]
load_balancer = WeightedRoundRobin(servers, weights)

for i in range(7):
    server = load_balancer.get_next_server()
    print(f"Request {i + 1} -> {server}")

Request 1 -> Server1
Request 2 -> Server1
Request 3 -> Server1
Request 4 -> Server1
Request 5 -> Server1
Request 6 -> Server2
Request 7 -> Server3


In [ ]:
CREATE DATABASE user_db_shard_1;
CREATE DATABASE user_db_shard_2;

USE user_db_shard_1;
CREATE TABLE users (
    id INT PRIMARY KEY,
    name VARCHAR(100)
);

USE user_db_shard_2;
CREATE TABLE users (
    id INT PRIMARY KEY,
    name VARCHAR(100)
);
